In [11]:
!pip install matplotlib 
!pip install sweetviz


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import os
import warnings

import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import sweetviz as sv

ARCHITETURE_PATH = "../data/merimee_acr.csv"
MUSEE_PATH = "../data/museofile.csv"
PATRIMOINE_PATH = "../data/museofile.csv" 

/Users/karimahallou/.pyenv/versions/3.10.12/envs/offre_culturelle/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df_archi = pd.read_csv(ARCHITETURE_PATH, sep="|", dtype=str, header=0)
df_archi.head(2)

,Reference_de_la_notice,Ancienne_reference_de_la_notice_RENV,Cadre_de_l_etude,Region,Numero_departement,Commune,Ancien_nom_commune,INSEE,Lieudit,Adresse_normalisee,...,Partie_d_elevation_exterieure,Materiaux_de_la_couverture,Typologie_de_plan,Technique_du_decor_porte_de_l_edifice,Indexation_iconographique_normalisee,Description_de_l_elevation_interieure,Emplacement_forme_structure_escalier,etat_de_conservation,Description_de_l_edifice,Commune_forme_editoriale
0,ACR0000001,NaN,Labellisé Architecture contemporaine remarquable,Auvergne-Rhône-Alpes,01,Ars-sur-Formans,NaN,01021,NaN,Docteur-Nodet (rue) 31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Située au sud de la basilique, l'église Notre-...",Ars-sur-Formans
1,ACR0000002,NaN,Labellisé Architecture contemporaine remarquable,Auvergne-Rhône-Alpes,01,Valserhône,Anciennement commune de : Bellegarde-sur-Valse...,01033,NaN,République (rue de la) 34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L'Hôtel de Ville de Bellegarde-sur-Valserine e...,Valserhône


In [7]:
df_musee = pd.read_csv(MUSEE_PATH, sep="|", dtype=str, header=0)
df_musee.head(2)

,Identifiant,Nom_officiel,Adresse,Lieu,Code_postal,Ville,Departement,Region,URL,Telephone,...,Themes,Artiste,Personnage_phare,Interet,Protection_batiment,Protection_espace,Refmer,Annee_creation,Date_de_mise_a_jour,Coordonnees
0,M1128,musée des sapeurs-pompiers de Lyon,358 avenue de Champagne,Centre de secours principal,69009,Lyon,Rhône,Auvergne-Rhône-Alpes,museepompiers.com/,04 72 17 54 54,...,"Armes, Uniformes, Véhicules pompiers, Archives...",NaN,NaN,NaN,NaN,NaN,SPR8400131,1971,2025-07-25,"45.790491, 4.797411"
1,M0001,musée de la Folie Marco,30 rue du docteur Sultzer,NaN,67140,Barr,Bas-Rhin,Grand Est,www.musee-foliemarco.com,03 88 08 94 72,...,"Arts décoratifs : Mobilier, Objets d'art",NaN,NaN,Demeure patricienne d'architecture française d...,Protégé au titre des Monuments historiques : h...,Abords d'un Monument historique. Site inscrit.,PA00084599;IA00115077,1960,2025-07-25,"48.410166, 7.451102"


In [6]:
df_patri = pd.read_csv(PATRIMOINE_PATH, sep="|", dtype=str, header=0)
df_patri.head(2)

,Identifiant,Nom_officiel,Adresse,Lieu,Code_postal,Ville,Departement,Region,URL,Telephone,...,Themes,Artiste,Personnage_phare,Interet,Protection_batiment,Protection_espace,Refmer,Annee_creation,Date_de_mise_a_jour,Coordonnees
0,M1128,musée des sapeurs-pompiers de Lyon,358 avenue de Champagne,Centre de secours principal,69009,Lyon,Rhône,Auvergne-Rhône-Alpes,museepompiers.com/,04 72 17 54 54,...,"Armes, Uniformes, Véhicules pompiers, Archives...",NaN,NaN,NaN,NaN,NaN,SPR8400131,1971,2025-07-25,"45.790491, 4.797411"
1,M0001,musée de la Folie Marco,30 rue du docteur Sultzer,NaN,67140,Barr,Bas-Rhin,Grand Est,www.musee-foliemarco.com,03 88 08 94 72,...,"Arts décoratifs : Mobilier, Objets d'art",NaN,NaN,Demeure patricienne d'architecture française d...,Protégé au titre des Monuments historiques : h...,Abords d'un Monument historique. Site inscrit.,PA00084599;IA00115077,1960,2025-07-25,"48.410166, 7.451102"


In [8]:
report_musee = sv.analyze(df_musee)
report_musee.show_html("sweetviz_report_musee.html")

Done! Use 'show' commands to display/save.   |██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| [100%]   00:00 -> (00:00 left)


Report sweetviz_report_musee.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [9]:
report_archi = sv.analyze(df_archi)
report_archi.show_html("sweetviz_report_archi.html")

Done! Use 'show' commands to display/save.   |██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| [100%]   00:00 -> (00:00 left)


Report sweetviz_report_archi.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [10]:
report_patri = sv.analyze(df_patri)
report_patri.show_html("sweetviz_report_patri.html")

Done! Use 'show' commands to display/save.   |██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| [100%]   00:00 -> (00:00 left)


Report sweetviz_report_patri.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
